In [1]:
!pip install imageio
!pip install imageio-ffmpeg
!pip install kmeans-pytorch
!wget https://people.eecs.berkeley.edu/~efros/img/Efros__photo_Peter_Badge_crop.jpg
!mv Efros__photo_Peter_Badge_crop.jpg efros.jpg

--2021-08-30 22:37:46--  https://people.eecs.berkeley.edu/~efros/img/Efros__photo_Peter_Badge_crop.jpg
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456031 (445K) [image/jpeg]
Saving to: ‘Efros__photo_Peter_Badge_crop.jpg’

Efros__photo_Peter_ 100%[===================>] 445.34K  --.-KB/s    in 0.1s    

2021-08-30 22:37:47 (4.34 MB/s) - ‘Efros__photo_Peter_Badge_crop.jpg’ saved [456031/456031]

calc_metrics.py
dataset_tool.py
dnnlib
Dockerfile
docker_run.sh
docs
e6MuEwjO9Xm9aILQc_UJg3pneDuV87-xIws_FgeiSIfgd9i510kbWwrvxgGgdroMqPyWQotseeQJX2vCA8XfU1tGzTN8bw
efros.jpg
Efros__photo_Peter_Badge_crop.jpg
environment.yml
generate.py
legacy.py
LICENSE.txt
malik.jpg
malik.jpg.1
metrics
out
playground.ipynb
projector.py
__pycache__
README.md
style_mixing.py
test.ipynb
torch_utils
training
train.py


In [4]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import os
from time import perf_counter
import click
import imageio
import numpy as np
import PIL.Image
import torch
import torch.nn.functional as F
import einops

import dnnlib
import legacy

from projector import project, project_blue, project_w_l2, sample_cluster_ws, project_cluster_reg

In [7]:
# network_pkl = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"
network_pkl = "https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-car-config-f.pkl"
target_fname = "efros.jpg"
outdir = "out"
save_video = True
seed = 202
num_steps = 200

In [8]:
np.random.seed(seed)
torch.manual_seed(seed)

In [9]:
# Load networks.
print('Loading networks from "%s"...' % network_pkl)
device = torch.device('cuda')
with dnnlib.util.open_url(network_pkl) as fp:
    G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device) # type: ignore

Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-car-config-f.pkl"...


In [10]:
# Load target image.
target_pil = PIL.Image.open(target_fname).convert('RGB')
w, h = target_pil.size
s = min(w, h)
target_pil = target_pil.crop(((w - s) // 2, (h - s) // 2, (w + s) // 2, (h + s) // 2))
target_pil = target_pil.resize((G.img_resolution, G.img_resolution), PIL.Image.LANCZOS)
target_uint8 = np.array(target_pil, dtype=np.uint8)

In [32]:
# Optimize projection.
start_time = perf_counter()
# projected_w_steps = project_blue(
#     G,
#     target=torch.cat((torch.zeros(2, 1024, 1024), 255*torch.ones(1, 1024, 1024)), 0), # pylint: disable=not-callable
#     num_steps=num_steps,
#     device=device,
#     verbose=True,
#     initial_noise_factor=0,
#     # initial_learning_rate=0
# )

# projected_w_steps = project(
#     G,
#     target=torch.cat((torch.zeros(2, 1024, 1024), 255*torch.ones(1, 1024, 1024)), 0), # pylint: disable=not-callable
#     num_steps=num_steps,
#     device=device,
#     verbose=True,
# )

projected_w_steps = project_w_l2(
    G,
    target=torch.cat((torch.zeros(2, 1024, 1024), 255*torch.ones(1, 1024, 1024)), 0), # pylint: disable=not-callable
    num_steps=num_steps,
    device=device,
    verbose=True,
    regularize_w_l2 = 0.05,
)

print (f'Elapsed: {(perf_counter()-start_time):.1f} s')

Computing W midpoint and stddev using 10000 samples...
tensor(0.8404, device='cuda:0', grad_fn=<SumBackward0>) tensor(0.1669, device='cuda:0', grad_fn=<AddBackward0>) 100000.0 tensor(0., device='cuda:0', grad_fn=<SumBackward0>) 0.05
step    1/200: dist 0.84 loss 16686.29
tensor(0.8136, device='cuda:0', grad_fn=<SumBackward0>) tensor(0.1809, device='cuda:0', grad_fn=<AddBackward0>) 100000.0 tensor(0., device='cuda:0', grad_fn=<SumBackward0>) 0.05
step    2/200: dist 0.81 loss 18095.54
tensor(0.7914, device='cuda:0', grad_fn=<SumBackward0>) tensor(0.1623, device='cuda:0', grad_fn=<AddBackward0>) 100000.0 tensor(0.0282, device='cuda:0', grad_fn=<SumBackward0>) 0.05
step    3/200: dist 0.79 loss 16226.94
tensor(0.7935, device='cuda:0', grad_fn=<SumBackward0>) tensor(0.1299, device='cuda:0', grad_fn=<AddBackward0>) 100000.0 tensor(0.1869, device='cuda:0', grad_fn=<SumBackward0>) 0.05
step    4/200: dist 0.79 loss 12993.66
tensor(0.8540, device='cuda:0', grad_fn=<SumBackward0>) tensor(0.0922

In [38]:
# Render debug output: optional video and projected image and W vector.
os.makedirs(outdir, exist_ok=True)
if save_video:
    video = imageio.get_writer(f'{outdir}/projblue_reg4.mp4', mode='I', fps=10, codec='libx264', bitrate='1M')
    print (f'Saving optimization progress video "{outdir}/proj.mp4"')
    for projected_w in projected_w_steps:
        synth_image = G.synthesis(projected_w.unsqueeze(0), noise_mode='const')
        synth_image = (synth_image + 1) * (255/2)
        synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()
        video.append_data(synth_image)
        # video.append_data(np.concatenate([target_uint8, synth_image], axis=1))
    video.close()

Saving optimization progress video "out/proj.mp4"


In [11]:
# Save final projected frame and W vector.
target_pil.save(f'{outdir}/target.png')
projected_w = projected_w_steps[-1]
synth_image = G.synthesis(projected_w.unsqueeze(0), noise_mode='const')
synth_image = (synth_image + 1) * (255/2)
synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()
PIL.Image.fromarray(synth_image, 'RGB').save(f'{outdir}/projblue.png')
np.savez(f'{outdir}/projected_wblue.npz', w=projected_w.unsqueeze(0).cpu().numpy())

In [40]:
from IPython.display import Video

Video("out/projblue_reg4.mp4")

In [11]:
# Generate clusters
clusters = sample_cluster_ws(
    G,
    num_clusters = 10,
    distance = 'euclidean',
    w_avg_samples = 10000,
    verbose = True,
    device=device,
)

Computing W midpoint and stddev using 10000 samples...
Setting up PyTorch plugin "bias_act_plugin"... Done.
running k-means on cuda..


[running kmeans]: 72it [00:15,  4.56it/s, center_shift=0.000000, iteration=72, tol=0.000100]


In [12]:
# Visualize clusters
for i in range(clusters.size()[0]):
    cluster_w = clusters[i,:].repeat([1, G.mapping.num_ws, 1]).to(device)
    synth_image = G.synthesis(cluster_w, noise_mode='const')
    synth_image = (synth_image + 1) * (255/2)
    synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()
    PIL.Image.fromarray(synth_image, 'RGB').save(f'{outdir}/carcluster{i}.png')

Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


In [17]:
projected_w_steps = project_cluster_reg(
    G,
    target=torch.cat((torch.zeros(2, 1024, 1024), 255*torch.ones(1, 1024, 1024)), 0), # pylint: disable=not-callable
    num_steps=num_steps,
    device=device,
    verbose=True,
    regularize_cluster_weight  = 0.05,
    regularize_w_clusters = clusters.to(device),
    use_avg_initialization=False,
)

Computing W midpoint and stddev using 10000 samples...
step    1/200: dist 0.86 loss 15416.03
step    2/200: dist 0.87 loss 19146.92
step    3/200: dist 0.84 loss 17197.07
step    4/200: dist 0.87 loss 13876.48
step    5/200: dist 0.88 loss 10234.90
step    6/200: dist 0.87 loss 7046.22
step    7/200: dist 0.82 loss 4895.28
step    8/200: dist 0.88 loss 3878.35
step    9/200: dist 0.79 loss 4162.96
step   10/200: dist 0.82 loss 5211.69
step   11/200: dist 0.79 loss 6411.40
step   12/200: dist 0.80 loss 6911.65
step   13/200: dist 0.79 loss 6515.99
step   14/200: dist 0.78 loss 5577.86
step   15/200: dist 0.77 loss 4825.45
step   16/200: dist 0.80 loss 4167.36
step   17/200: dist 0.76 loss 3492.12
step   18/200: dist 0.73 loss 2583.55
step   19/200: dist 0.69 loss 1767.73
step   20/200: dist 0.77 loss 1263.82
step   21/200: dist 0.78 loss 1131.15
step   22/200: dist 0.76 loss 1292.29
step   23/200: dist 0.78 loss 1634.92
step   24/200: dist 0.75 loss 2028.75
step   25/200: dist 0.70 los

In [18]:
# Render debug output: optional video and projected image and W vector.
os.makedirs(outdir, exist_ok=True)
if save_video:
    video = imageio.get_writer(f'{outdir}/projblue_clusterreg5.mp4', mode='I', fps=10, codec='libx264', bitrate='1M')
    for projected_w in projected_w_steps:
        synth_image = G.synthesis(projected_w.unsqueeze(0), noise_mode='const')
        synth_image = (synth_image + 1) * (255/2)
        synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()
        video.append_data(synth_image)
        # video.append_data(np.concatenate([target_uint8, synth_image], axis=1))
    video.close()

In [19]:
from IPython.display import Video

Video("out/projblue_clusterreg5.mp4")

In [36]:
n_samples = 4
n_grid = 2
samples = []

for i in range(n_samples):
    # projected_w_steps = project(
    #     G,
    #     target=torch.cat((torch.zeros(2, target_uint8.shape[0], target_uint8.shape[1]), 255*torch.ones(1, target_uint8.shape[0], target_uint8.shape[1])), 0), # pylint: disable=not-callable
    #     num_steps=num_steps,
    #     device=device,
    #     verbose=True,
    #     use_avg_initialization=False,
    # )

    projected_w_steps = project_w_l2(
        G,
        target=torch.cat((torch.zeros(2, target_uint8.shape[0], target_uint8.shape[1]), 255*torch.ones(1, target_uint8.shape[0], target_uint8.shape[1])), 0), # pylint: disable=not-callable
        num_steps=num_steps,
        device=device,
        verbose=True,
        regularize_w_l2 = 0.05,
        use_avg_initialization=False,
    )

    # projected_w_steps = project_cluster_reg(
    #     G,
    #     target=torch.cat((torch.zeros(2, target_uint8.shape[0], target_uint8.shape[1]), 255*torch.ones(1, target_uint8.shape[0], target_uint8.shape[1])), 0), # pylint: disable=not-callable
    #     num_steps=num_steps,
    #     device=device,
    #     verbose=True,
    #     regularize_cluster_weight  = 0.05,
    #     regularize_w_clusters = clusters.to(device),
    #     use_avg_initialization=False,
    # )
    samples.append(projected_w_steps)

videos = torch.zeros(n_samples, samples[0].size()[0], 3, target_uint8.shape[0], target_uint8.shape[1])

for i in range(n_samples):
    projected_w_steps = samples[i]
    frame = 0
    for projected_w in projected_w_steps:
        synth_image = G.synthesis(projected_w.unsqueeze(0), noise_mode='const')
        synth_image = (synth_image + 1) * (255/2)
        synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0]
        videos[i, frame] = synth_image.permute(2, 0, 1)
        frame += 1

grid_videos = einops.rearrange(videos, "(n1 n2) t c h w -> t (n1 h) (n2 w) c", n1=n_grid)

# Render debug output: optional video and projected image and W vector.
os.makedirs(outdir, exist_ok=True)
video = imageio.get_writer(f'{outdir}/project_bluecar_noreg_random.mp4', mode='I', fps=10, codec='libx264', bitrate='1M')
for frame in grid_videos:
    video.append_data(frame.cpu().numpy())
    # video.append_data(np.concatenate([target_uint8, synth_image], axis=1))
video.close()

Computing W midpoint and stddev using 10000 samples...
step    1/200: dist 0.78 loss 14744.19
step    2/200: dist 0.82 loss 16073.50
step    3/200: dist 0.83 loss 14420.26
step    4/200: dist 0.79 loss 11639.06
step    5/200: dist 0.79 loss 8731.59
step    6/200: dist 0.82 loss 6500.27
step    7/200: dist 0.75 loss 5379.22
step    8/200: dist 0.83 loss 4759.15
step    9/200: dist 0.76 loss 4353.06
step   10/200: dist 0.74 loss 3549.04
step   11/200: dist 0.85 loss 2677.29
step   12/200: dist 0.78 loss 2173.22
step   13/200: dist 0.76 loss 2641.97
step   14/200: dist 0.72 loss 3479.51
step   15/200: dist 0.73 loss 3990.03
step   16/200: dist 0.80 loss 3749.00
step   17/200: dist 0.67 loss 3046.16
step   18/200: dist 0.79 loss 2139.86
step   19/200: dist 0.74 loss 1282.00
step   20/200: dist 0.70 loss 691.38
step   21/200: dist 0.72 loss 551.08
step   22/200: dist 0.74 loss 874.99
step   23/200: dist 0.73 loss 1342.96
step   24/200: dist 0.69 loss 1631.79
step   25/200: dist 0.70 loss 15

Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
